In [0]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

#event hub connection string
connection_string = (
    "Endpoint=sb://fraud-detection-project-ns.servicebus.windows.net/;"
    "SharedAccessKeyName=RootManageSharedAccessKey;"
    "SharedAccessKey=jqE/NKbtmuz6h3cAc1PfBPai8DUkWdnHt+AEhEvO9dQ=;"
    "EntityPath=transaction-streams"
)

eventhubs_conf = {
    'eventhubs.connectionString': sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connection_string)
}

#transaction schema
schema = StructType([
    StructField("transaction_id", StringType()),
    StructField("user_id", StringType()),
    StructField("amount", DoubleType()),
    StructField("merchant_id", StringType()),
    StructField("location", StringType()),
    StructField("timestamp", StringType())
])

#read stream from event hub
raw_stream = (
    spark.readStream
    .format("eventhubs")
    .options(**eventhubs_conf)
    .load()
)

#flatten data
parsed_stream = (
    raw_stream
    .withColumn("body", col("body").cast("string"))
    .select(from_json(col("body"), schema).alias("data"))
    .select("data.*")
)

#mount container
spark.conf.set(
  "fs.azure.account.key.storageaccount9613.dfs.core.windows.net",
  "XvCGvmbsEeJ4hjnQ7D3xUAxNMQznhaW6o8UhOXpBTFmxxOZtPhHenywE2Z3qmUYlup6bazd/9glE+AStAsXwjA=="
)

#write transactions to storage account
query = (
    parsed_stream.writeStream
    .format("delta")
    .option("checkpointLocation", "abfss://fraud-detection-container@storageaccount9613.dfs.core.windows.net/checkpoints/fraud")
    .start("abfss://fraud-detection-container@storageaccount9613.dfs.core.windows.net/fraud_data")
)
